# Applying Machine Learning and Deep Learning to identify home appliances consuming excess power

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [1]:
# Home Appliances Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import Dataset
df_home_data_test = pd.read_csv('../../data/home_data_predict.csv')
df_home_priority = pd.read_csv('../../data/home_priority.csv')
date_time_group = pd.read_csv('../../data/date_time_group.csv')

In [3]:
df_home_data_test.head()

,device,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power
0,AC,100,hot,2018-01-01,00:00:00,01:00:00,midnight,3,21,1528.898438
1,AC,100,very cold,2018-01-01,00:00:00,01:00:00,midnight,10,5,1512.648438
2,AC,100,cold,2018-01-01,02:00:00,03:00:00,midnight,4,22,1521.007812
3,AC,100,cold,2018-01-01,03:00:00,04:00:00,midnight,3,5,1499.714844
4,AC,100,very cold,2018-01-01,05:00:00,06:00:00,early morning,2,35,1507.480469


In [4]:
df_home_priority.head()

,device,weather_type,time,priority,order
0,AC,low cold,early morning,8,52
1,AC,low cold,morning,8,49
2,AC,low cold,afternoon,8,48
3,AC,low cold,evening,8,50
4,AC,low cold,night,8,51


In [5]:
date_time_group.head()

,date,from_time,power
0,2018-01-01,00:00:00,10372
1,2018-01-01,01:00:00,9393
2,2018-01-01,02:00:00,10013
3,2018-01-01,03:00:00,10171
4,2018-01-01,04:00:00,10650


In [6]:
total_power_consumption = df_home_data_test.groupby(['date', 'from_time']).sum()['power'].reset_index()

In [7]:
sample_date = '2018-01-03'
# sample_date_format = '18-01-2018'
sample_time = '05:00:00'

In [8]:
sample_df = df_home_data_test[(df_home_data_test['date'] == sample_date) & (df_home_data_test['from_time'] == sample_time)]

sample_df_priority = df_home_priority[(df_home_priority['device'] == 'AC') & (df_home_priority['weather_type'] == 'low cold') & (df_home_priority['time'] == 'early morning')]

sample_total_power = sample_df['power'].sum()

In [9]:
power = date_time_group[(date_time_group['date'] == sample_date) & (date_time_group['from_time'] == sample_time)]
condition = sample_total_power > power['power'][power['power'].index[0]]

In [10]:
priorities = []

In [11]:
for index, row in sample_df.iterrows():
#     print(row['device'], row['weather_type'], row['time'])
    priorities.append(df_home_priority[(df_home_priority['device'] == row['device']) 
                           & (df_home_priority['weather_type'] == row['weather_type']) 
                           & (df_home_priority['time'] == row['time'])].values[0][4])

In [12]:
priorities = pd.Series(priorities)

sample_df = sample_df.assign(priorities=priorities.values)
sample_df = sample_df.sort_values(['priorities', 'no_of_people'], ascending=[True,False])
old_df = sample_df.copy()
sample_after_power = 10000000

In [13]:
while sample_total_power < sample_after_power:
    no_of_people_index = sample_df.iloc[[-2]]['no_of_people'].index[0]

    last_one_room = sample_df.iloc[[-2]]['room'].values[0]
    last_no_of_people = sample_df.iloc[[-1]]['no_of_people'].values[0]
    last_one_no_of_people = sample_df.iloc[[-2]]['no_of_people'].values[0]
    last_room = sample_df.iloc[[-1]]['room'].values[0]

    sample_df.at[no_of_people_index, 'no_of_people'] = last_no_of_people + last_one_no_of_people 

    sample_df.drop(sample_df.tail(1).index,inplace=True) 

    print('Moving ' + str(last_no_of_people) + ' people from ' + str(last_room) + ' to ' + str(last_one_room))

    sample_after_power = sample_total_power - sample_df['power'].sum()
    print('Saves ' + str(sample_after_power) + ' of' + ' electricity')

    print('Power consumption reduced from ' +  str(sample_total_power) + ' to ' + str(sample_total_power - sample_after_power))

Moving 6 people from 118 to 102
Saves 1473.16015625 of electricity
Power consumption reduced from 10338.1015625 to 8864.94140625


In [14]:
sample_df

,device,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power,priorities
825,AC,103,very hot,2018-01-03,05:00:00,06:00:00,early morning,15,55,1484.207031,17
1067,AC,104,very hot,2018-01-03,05:00:00,06:00:00,early morning,3,16,1466.906250,17
4966,AC,119,very hot,2018-01-03,05:00:00,06:00:00,early morning,1,40,1505.234375,17
1068,AC,104,low hot,2018-01-03,05:00:00,06:00:00,early morning,8,35,1471.636719,41
2195,AC,108,low hot,2018-01-03,05:00:00,06:00:00,early morning,0,0,1489.589844,41
525,AC,102,cold,2018-01-03,05:00:00,06:00:00,early morning,19,59,1447.367188,70


In [15]:
old_df

,device,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power,priorities
825,AC,103,very hot,2018-01-03,05:00:00,06:00:00,early morning,15,55,1484.207031,17
1067,AC,104,very hot,2018-01-03,05:00:00,06:00:00,early morning,3,16,1466.906250,17
4966,AC,119,very hot,2018-01-03,05:00:00,06:00:00,early morning,1,40,1505.234375,17
1068,AC,104,low hot,2018-01-03,05:00:00,06:00:00,early morning,8,35,1471.636719,41
2195,AC,108,low hot,2018-01-03,05:00:00,06:00:00,early morning,0,0,1489.589844,41
525,AC,102,cold,2018-01-03,05:00:00,06:00:00,early morning,13,59,1447.367188,70
4710,AC,118,cold,2018-01-03,05:00:00,06:00:00,early morning,6,27,1473.160156,70
